In [1]:
from collections import Counter
from math import log2, inf
import random
import time
from tqdm import tqdm

In [2]:
valid_words = []
answer_words = []
with open ("wordle-allowed-guesses.txt") as f:
    for line in f:
        valid_words.append(line.strip())
        
with open ("wordle-answers-alphabetical.txt") as f:
    for line in f:
        answer_words.append(line.strip())

In [3]:
cache = {
    "~~~~~" : "clint",
    "~~~~*" : "guilt",
    "~~~~?" : "denet",
    "~~~*~" : "fitch",
    "~~~**" : "abets",
    "~~~*?" : "feted",
    "~~~?~" : "glint",
    "~~~?*" : "pudic",
    "~~~??" : "tined",
    "~~*~~" : "clink",
    "~~*~*" : "glitz",
    "~~*~?" : "depth",
    "~~**~" : "dhuti",
    "~~***" : "abaft",
    "~~**?" : "lathy",
    "~~*?~" : "clint",
    "~~*?*" : "diact",
    "~~*??" : "ached",
    "~~?~~" : "clint",
    "~~?~*" : "gault",
    "~~?~?" : "natal",
    "~~?*~" : "humic",
    "~~?**" : "afire",
    "~~?*?" : "altho",
    "~~??~" : "riyal",
    "~~??*" : "barca",
    "~~???" : "talar",
    "~*~~~" : "culty",
    "~*~~*" : "pugil",
    "~*~~?" : "meynt",
    "~*~*~" : "almud",
    "~*~?~" : "cyton",
    "~*~?*" : "faugh",
    "~*~??" : "rewth",
    "~**~~" : "loamy",
    "~***~" : "board",
    "~**?~" : "roach",
    "~*?~~" : "liman",
    "~*?*~" : "cobra",
    "~*??~" : "malty",
    "~?~~~" : "clint",
    "~?~~*" : "bling",
    "~?~~?" : "lento",
    "~?~*~" : "acidy",
    "~?~**" : "chore",
    "~?~*?" : "metro",
    "~?~?~" : "cutin",
    "~?~?*" : "pownd",
    "~?~??" : "trued",
    "~?*~~" : "piano",
    "~?*~*" : "ovate",
    "~?**~" : "ovary",
    "~?*?~" : "bravo",
    "~??~~" : "cloot",
    "~??~*" : "bundt",
    "~??~?" : "cameo",
    "~??*~" : "abaca",
    "~??**" : "adore",
    "~??*?" : "opera",
    "~???~" : "maron",
    "*~~~~" : "thilk",
    "*~~~*" : "pling",
    "*~~~?" : "clipt",
    "*~~*~" : "hault",
    "*~~**" : "shire",
    "*~~*?" : "sperm",
    "*~~?~" : "butch",
    "*~~?*" : "apeek",
    "*~~??" : "newie",
    "*~*~~" : "thilk",
    "*~*~*" : "thilk",
    "*~**~" : "chapt",
    "*~***" : "chapt",
    "*~*?~" : "stair",
    "*~?~~" : "dault",
    "*~?~*" : "sauce",
    "*~?~?" : "dempt",
    "*~??~" : "cuppy",
    "*~???" : "enmew",
    "**~~~" : "atony",
    "**~~*" : "solve",
    "**~*~" : "sorry",
    "**~??" : "sober",
    "***~~" : "soapy",
    "**??~" : "solar",
    "*?~~~" : "cloot",
    "*?~~*" : "knelt",
    "*?~*~" : "nicht",
    "*?~**" : "chapt",
    "*?~?~" : "scour",
    "*??~~" : "salon",
    "*???~" : "savor",
    "?~~~~" : "hists",
    "?~~~*" : "cunit",
    "?~~~?" : "teugh",
    "?~~*~" : "usurp",
    "?~~?~" : "cruft",
    "?~~?*" : "centu",
    "?~~??" : "richt",
    "?~*~~" : "chals",
    "?~*~*" : "leuch",
    "?~*~?" : "bufty",
    "?~*?~" : "bachs",
    "?~*?*" : "erase",
    "?~?~~" : "linty",
    "?~?~*" : "thump",
    "?~?~?" : "ajwan",
    "?~??~" : "harsh",
    "?~??*" : "arise",
    "?*~~~" : "fubsy",
    "?*~~*" : "pilum",
    "?*~~?" : "nosey",
    "?*~?~" : "roost",
    "?*~?*" : "horse",
    "?*~??" : "loser",
    "?**~~" : "abaca",
    "?**?~" : "roast",
    "??~~~" : "agloo",
    "??~~*" : "aitch",
    "??~~?" : "onset",
    "??~?~" : "acids",
    "??~?*" : "prose",
    "??~??" : "verso",
    "??*~~" : "chaos",
    "???~~" : "ascot",
    "????~" : "arson",
    "????*" : "arose"
}

In [4]:
def guess(correct_word, guess_word):
    assert guess_word in valid_words or guess_word in answer_words, "invalid word"
    rtn = ''
    guess_word_letter_counts = Counter()
    correct_word_letter_counts = Counter(correct_word)
    for idx, letter in enumerate(guess_word):
        guess_word_letter_counts[letter] += 1
        if correct_word[idx] == letter:
            rtn += '*'
        elif letter in correct_word and \
            correct_word_letter_counts[letter] >= guess_word_letter_counts[letter]:
                rtn += '?'
        else:
            rtn +='~'
    return rtn

class Info_Gain_Solver():
    
    def __init__(self, print_answers=True):
        self.reset()
        self.print_answers = print_answers
        
    def next_word(self, result=None, guess_word=None):
        guess_word = guess_word if guess_word else self.prev_suggestion
        
        if result:
            self.apply_rule(result, guess_word)
        
        #use cached results for first two guesses
        if len(self.words) == 2315:
            self.prev_suggestion = "soare"
            return "soare"
        elif self.prev_word_len == 2315 and self.prev_word == "soare":
            self.prev_suggestion = cache[result]
            return cache[result]
        
        best, best_word = inf, None
        for word in self.words:
            if best == 0:
                break
            e_entropy = self.expected_entropy(word)
            if e_entropy < best:
                best, best_word = e_entropy, word
                    
        for word in valid_words + self.incorrect_words:
            if best == 0:
                break
            e_entropy = self.expected_entropy(word)
            if e_entropy < best:
                best, best_word = e_entropy, word
        self.prev_suggestion = best_word
        return best_word
        
    def expected_entropy(self, guess_word):
        starting_entropy = log2(len(self.words))
        results = Counter()
        for word in self.words:
            answer = guess(word, guess_word)
            results[answer] += 1
        expected_entropy = 0
        for _, count in results.items():
            expected_entropy += count * log2(count)
        expected_entropy /= len(self.words)
        return expected_entropy
        
    def apply_rule(self, result, guess_word):
        
        self.prev_word_len = len(self.words)
        new_words = []
        for word in self.words:
            if guess(word, guess_word) == result:
                new_words.append(word)
            else:
                self.incorrect_words.append(word)

        self.words = new_words
        self.prev_word = guess_word
        if self.print_answers and len(new_words) <= 10:
            print("possible words: ")
            for word in new_words:
                print(word)
            print()
            
    def reset(self):
        self.words = answer_words[:]
        self.incorrect_words = []
        self.prev_word = None
        self.prev_suggestion = None
            

In [5]:
def test(word=None, print_guesses=False):
    start_time = time.time()
    if not word:
        word = random.choice(answer_words)
    solver = Info_Gain_Solver(print_answers=False)
    i = 0
    solved = False
    result = None
    while not solved:
        guess_word = solver.next_word(result)
        result = guess(word, guess_word)
        solved = result == "*****"
        if print_guesses:
            print(guess_word, result)
        i += 1
    return i, time.time() - start_time
    

def test_n(n, print_guesses = False):
    # if n == 2315, test all words
    # otherwise, test n random words
    results = []
    times = []
    for i in tqdm(range(n)):
        word = answer_words[i] if n == 2315 else None
        result, time_taken = test(word, print_guesses=print_guesses)
        results.append(result)
        times.append(time_taken)
    print(f"average number of guesses {sum(results)/n}")
    print(f"max number of guesses {max(results)}")
    print(f"average computation time {sum(times)/n}")
    print(f"max computation time {max(times)}")

In [6]:
test(word = "wince", print_guesses=True)

soare ~~~~*
guilt ~~?~~
mince ~****
wince *****


(4, 0.26743316650390625)

In [30]:
test_n(2315)

100%|█████████████████████████████████████| 2315/2315 [1:10:15<00:00,  1.82s/it]

average number of guesses 3.472570194384449
max number of guesses 6
average computation time 1.8202573021338775
max computation time 34.74883818626404
